In [ ]:
#### CSE 587: Project 3
#### Members: Thankam Abish, Krishna Menon, Lucas Bietzel

In [ ]:
import findspark

In [ ]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [ ]:
import pyspark

In [ ]:
sc = pyspark.SparkContext()

In [ ]:
from pyspark.sql import *

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Pyspark demo") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
###ADJUST TO CORRECT FILE PATHS

dfs = spark.read.csv('/home/cse587/A3Data/train.csv', escape='"', inferSchema = 'True',header="True")
testdfs = spark.read.csv('/home/cse587/A3Data/test.csv', escape='"', inferSchema = 'True',header="True")

In [ ]:
dfs.show()

In [ ]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

### SMALL DATAFRAME USED FOR TESTING
#dfs = spark.createDataFrame([
 #   (12340, "Hi hi HI HI  , HI HI", "['Drama','Comedy']"),
  #  (123545, "I wish Java could use case classes", "['Drama','Comedy','Action']"),
   # (2123, "things are really neat. wow", "['Action','Comedy','Horror']"),
    #(21523, "fat cats are cute", "['Action','Comedy','Horror']")], 
    #["id", "plot", "genre"])
    
#testdfs = spark.createDataFrame([
 #   (12640, "Hi wish cats")],
  #  ["id", "plot"])

In [ ]:
#### FIND BAD DATA

def to_fix(x):
    try:
        eval(x)
        return True
    except:
        return False


fix_udf = udf(lambda x: to_fix(x), BooleanType())
dfs2 = dfs.withColumn('validity', fix_udf('genre'))
dfs2.show()



In [ ]:
dfs = dfs2.filter(col("validity")==True)
dfs.count()

In [ ]:
dfs = dfs.dropna()
dfs.count()

In [ ]:
#plot text cleaning, shows the clean text 
def clean(x):
    dfs2 = x.withColumn('plot', (col('plot')).cast(StringType()))
    text = regexp_replace(dfs2['plot'], "[^a-zA-Z ]","")
    text = regexp_replace(text, ' +', ' ')
    dfs2 = dfs2.withColumn('plot', text)
    dfs2.show()
    dfs2 = dfs2.withColumn('plot', lower(col('plot')))
    dfs2.show()
    c = split(dfs2['plot'],' ')
    dfs2 = dfs2.withColumn('plot', c)
    dfs2.show()

    ##stop word removal
    remover = StopWordsRemover(inputCol="plot", outputCol="filtered")
    dfs2 = remover.transform(dfs2)
    dfs2 = dfs2.withColumn('plot', col('filtered'))
    dfs2 = dfs2.drop('filtered')
    return dfs2

dfs2 = clean(dfs2)
testdfs = clean(testdfs)

In [ ]:
dfs2.printSchema()
dfs2.show()
testdfs.printSchema()
dfs2.show()

In [ ]:
####################################################################
############     PART 1   (ONLY RUN ONE PART)
####################################################################
cv = CountVectorizer(inputCol="plot", outputCol="features")
model = cv.fit(dfs2)
dfs3 = model.transform(dfs2)
testdfs3 = model.transform(testdfs)

In [ ]:
####################################################################
############     PART 2    (ONLY RUN ONE PART)
####################################################################
cv = CountVectorizer(inputCol="plot", outputCol="rawFeatures")
model = cv.fit(dfs2)
dfs3 = model.transform(dfs2)
testdfs3 = model.transform(testdfs)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(dfs3)
dfs3 = idfModel.transform(dfs3)
testdfs3 = idfModel.transform(testdfs3)

In [ ]:
####################################################################
############     PART 3    (ONLY RUN ONE PART)
####################################################################
word2Vec = Word2Vec(vectorSize = 100, inputCol="plot", outputCol="features")
w2vmodel = word2Vec.fit(dfs2)
dfs3 = w2vmodel.transform(dfs2)
testdfs3 = w2vmodel.transform(testdfs)

In [ ]:
dfs3.printSchema()
dfs3.count()
testdfs3.printSchema()
dfs3.show()
testdfs3.show()

In [ ]:
#### DETECT BAD DATA

def to_fix(x): 
    try:
        eval(x)
        return True
    except:
        return False

fix_udf = udf(lambda x: to_fix(x), BooleanType())
dfs4 = dfs3.withColumn('validity', fix_udf('genre'))
dfs4.show()

dfs3 = dfs4.filter(col("validity")==True)

In [ ]:
#### CREATE BINARY LABEL COLUMNS FOR GENRE

genres = ['Drama','Comedy','Romance Film','Thriller','Action',
          'World cinema','Crime Fiction','Horror','Black-and-white',
          'Indie','Action/Adventure','Adventure','Family Film',
          'Short Film','Romantic drama','Animation','Musical',
          'Science Fiction','Mystery','Romantic comedy']

def to_genre(x):
    a = eval(x)
    b = []
    for i in genres:
        if i in a:
            b.append(1)
        else:
            b.append(0)
    return b

to_list_udf = udf(lambda x: to_genre(x), ArrayType(IntegerType()))
dfs6 = dfs3.withColumn('genre', to_list_udf('genre'))
dfs6.show()

In [ ]:
#### SEPARATED GENRE COLUMNS

for i in range(len(genres)):
    dfs6=dfs6.withColumn(genres[i],col('genre').getItem(i))
dfs6 = dfs6.drop('genre')
dfs6.show()

In [ ]:
dfs6.printSchema()
dfs6.show()

In [ ]:
# SINGLE MODEL CREATION
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'Drama', maxIter=100)
lrModel = lr.fit(dfs6)

In [ ]:
# THE TESTING FOR SINGLE MODEL CREATION
prediction = lrModel.transform(testdfs3)
prediction.show()

In [ ]:
# ALL 20 BINARY CLASSIFIERS
import numpy as np
from pyspark.ml.classification import LogisticRegression
biglr = np.array([])
biglrModel = np.array([])
for i in range(len(genres)):
    biglr = np.append(biglr,LogisticRegression(featuresCol = 'features', labelCol = genres[i], maxIter=100))
    biglrModel = np.append(biglrModel, biglr[i].fit(dfs6))

In [ ]:
# PREDICTIONS FROM BINARY CLASSIFIERS
preds = np.array([])
for i in range(len(genres)):
    a = biglrModel[i].transform(testdfs3)
    a = a.withColumn('prediction', col('prediction').cast(IntegerType()))
    preds = np.append(preds, a)
    preds[i].show()

In [ ]:
### MERGE PREDICTIONS
final = preds[0].withColumn('id',col('movie_id').cast(StringType())).select(col('id'))
final.show()

for i in range(len(genres)):
    
    final = final.join(preds[i], final.id == preds[i].movie_id)
    final = final.withColumn(genres[i],col('prediction'))
    final = final.drop('movie_name','movie_id','plot','features','rawPrediction','probability','prediction','rawFeatures')
    final.show()

df = final.withColumn('prediction', 
                    concat(col('Drama')
                          ,lit(' '),col('Comedy')
                          ,lit(' '),col('Romance Film')
                          ,lit(' '),col('Thriller')
                          ,lit(' '),col('Action')
                          ,lit(' '),col('World cinema')
                          ,lit(' '),col('Crime Fiction')
                          ,lit(' '),col('Horror')
                          ,lit(' '),col('Black-and-white')
                          ,lit(' '),col('Indie')
                          ,lit(' '),col('Action/Adventure')
                          ,lit(' '),col('Adventure')
                          ,lit(' '),col('Family Film')
                          ,lit(' '),col('Short Film')
                          ,lit(' '),col('Romantic drama')
                          ,lit(' '),col('Animation')
                          ,lit(' '),col('Musical')
                          ,lit(' '),col('Science Fiction')
                          ,lit(' '),col('Mystery')
                          ,lit(' '),col('Romantic comedy')))
df.show()

In [ ]:
### PREPARING FORMAT FOR CSV FILE

convertToCSV = df.select(col('id').alias('movie_id'),col('prediction'))
convertToCSV.show()

In [ ]:
### WRITE TO CSV
convertToCSV = convertToCSV.withColumn('movie_id', col('movie_id').cast(IntegerType()))
convertToCSV.printSchema()
convertToCSV.write.csv('Part2.csv')